Bi-LSTM Code

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import nltk
nltk.download('punkt_tab')
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional
from tensorflow.keras.models import Model
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/REVIEW/Pheme_dataset_balanced.csv')
df.value_counts()
df['is_rumor'] = pd.to_numeric(df['is_rumor'], errors='coerce')
df.isnull().sum()
df = df.dropna()
df.info()
df['text'][0]

Preprocessing the Data

In [ ]:
nltk.download("stopwords")
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in stop_words:
            result.append(token)
    return result
df['clean_joined'] = df['text'].apply(lambda x: " ".join(preprocess(x)))
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)  
vectorizer.fit(df['clean_joined'])
tfidf_matrix = vectorizer.transform(df['clean_joined'])
feature_names = vectorizer.get_feature_names_out()
features_df = pd.DataFrame(feature_names, columns=["Features"])
print(features_df)
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(feature_names)
N = 10  
sequences = []
for row in tfidf_matrix:
    top_feature_indices = row.toarray().argsort()[0, -N:][::-1]
    top_feature_tokens = [feature_names[i] for i in top_feature_indices]
    sequence = tokenizer.texts_to_sequences(top_feature_tokens)
    sequence = [item for sublist in sequence for item in sublist]
    sequences.append(sequence)
from tensorflow.keras.preprocessing.sequence import pad_sequences 
padded_sequences = pad_sequences(sequences, maxlen=40, padding='post', truncating='post')
df['clean'] = df['text'].apply(preprocess)
df['text'][0]
print(df['clean'][0])
df.head()
list_of_words = []
for i in df.clean:
    for j in i:
        list_of_words.append(j)
total_words = len(list(set(list_of_words)))
total_words
df['clean_joined'] = df['clean'].apply(lambda x: " ".join(x))
plt.figure(figsize = (8, 8))
sns.countplot(y = "topic", data = df)
plt.figure(figsize = (8, 8))
sns.countplot(y = "is_rumor", data = df)
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 2000 , width = 1600 , height = 800 , stopwords = stop_words).generate(" ".join(df[df['is_rumor'] == 1]['clean_joined'].astype(str)))
plt.imshow(wc, interpolation = 'bilinear')
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 2000 , width = 1600 , height = 800 , stopwords = stop_words).generate(" ".join(df[df['is_rumor'] == 0]['clean_joined'].astype(str)))
plt.imshow(wc, interpolation = 'bilinear')
maxlen = -1
for doc in df.clean_joined:
    tokens = nltk.word_tokenize(doc)
    if(maxlen<len(tokens)):
        maxlen = len(tokens)
print("The maximum number of words in any document is =", maxlen)
import plotly.express as px  
word_counts = [len(nltk.word_tokenize(x)) for x in df.clean_joined]
fig = px.histogram(x=word_counts, nbins=100)
fig.update_layout(
    title="Distribution of Word Counts in News Articles",
    xaxis_title="Number of Words per Article",
    yaxis_title="Frequency",
)
fig.show()
padded_sequences = pad_sequences(sequences, maxlen=40, padding='post', truncating='post')
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
embedding_index = {}
with open('/content/drive/My Drive/Colab Notebooks/REVIEW/glove.6B.100d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs
embedding_dim = 100
total_words = len(tokenizer.word_index) + 1  
embedding_matrix = np.zeros((total_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < total_words:  # Ensure we stay within bounds
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

Training the Model

In [ ]:
from sklearn.model_selection import train_test_split
df['is_rumor'] = pd.to_numeric(df['is_rumor'], errors='coerce').fillna(0).astype(int)
x_train, x_test, y_train, y_test = train_test_split(padded_sequences, df['is_rumor'].astype(int), test_size=0.2)
from keras.regularizers import l2
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=embedding_dim, weights=[embedding_matrix], trainable=True))  # Set trainable=True
model.add(Bidirectional(LSTM(128, return_sequences=False)))  # Set return_sequences=False
model.add(Dropout(0.5))  # Dropout rate of 50%
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))   
model.add(Dropout(0.5))  
model.add(Dense(1, activation='sigmoid'))  
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()
y_train = np.asarray(y_train)
model.fit(x_train, y_train, batch_size=64, validation_split=0.1, epochs=50)

pred = model.predict(x_test)
prediction = []
for i in range(len(pred)):
    if pred[i].item() > 0.5:
        prediction.append(1)
    else:
        prediction.append(0)

Calculating the Accuracy

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import precision_score, f1_score
import pandas as pd
y_test_list = y_test.dropna().tolist()  # Remove NaN values
prediction = prediction[:len(y_test_list)]
accuracy = accuracy_score(y_test_list, prediction)
print("Model Accuracy : ", accuracy)
print(classification_report(y_test_list, prediction))
precision = precision_score(y_test_list, prediction)
print("Precision:", precision)
f1 = f1_score(y_test_list, prediction)
print("F1-Score:", f1)
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, precision_score, f1_score
import matplotlib.pyplot as plt
pred = model.predict(x_test)
prediction = []
for i in range(len(pred)):
    if pred[i].item() > 0.5:
        prediction.append(1)
    else:
        prediction.append(0)

Confusion Matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
predictions = [1 if prob > 0.5 else 0 for prob in pred]
cm = confusion_matrix(y_test, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Not Rumor", "Rumor"])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix for Bi-LSTM Model")
plt.show()

Identify False Positive and Negative

In [ ]:
false_positives = []
false_negatives = []
for i in range(len(y_test_list)):
    if prediction[i] == 1 and y_test_list[i] == 0:
        false_positives.append((i, x_test[i]))  
    elif prediction[i] == 0 and y_test_list[i] == 1:
        false_negatives.append((i, x_test[i]))  
def remove_padding(sequence):
    return sequence[sequence != 0]
print(f"\nTotal False Positives: {len(false_positives)}")
print(f"Total False Negatives: {len(false_negatives)}")
print("\nFalse Positives:")
for index, seq in false_positives:
    cleaned_sequence = remove_padding(seq)  
    print(f"Index: {index}, Predicted: {prediction[index]}, Actual: {y_test_list[index]}, Sequence: {cleaned_sequence}")
print("\nFalse Negatives:")
for index, seq in false_negatives:
    cleaned_sequence = remove_padding(seq)  
    print(f"Index: {index}, Predicted: {prediction[index]}, Actual: {y_test_list[index]}, Sequence: {cleaned_sequence}")
num_false_positives = len(false_positives)
num_false_negatives = len(false_negatives)
labels = ['False Positives', 'False Negatives']
counts = [num_false_positives, num_false_negatives]
plt.figure(figsize=(8, 5))
plt.bar(labels, counts, color=['blue', 'red'])
plt.title('Count of False Positives and False Negatives')
plt.ylabel('Count')
plt.xlabel('Classification Error Type')
plt.ylim(0, max(counts) + 100)
offset = 10
for i in range(len(counts)):
    plt.text(i, counts[i] + offset, str(counts[i]), ha='center')

plt.show()

Generate random news articles for testing

In [ ]:
import random
def generate_random_news(num_articles=5):
    # Sample words to create random sentences
    sample_words = ["breaking", "news", "report", "claims", "study", "research", "findings",
                    "investigation", "reveals", "analysis", "government", "officials",
                    "says", "sources", "data", "evidence", "public", "concerns", "murder",
                    "scandal", "robbery", "bribes", "declares", "war"]

    random_articles = []
    for _ in range(num_articles):
        sentence_length = random.randint(5, 15)
        random_sentence = ' '.join(random.choices(sample_words, k=sentence_length))
        random_articles.append(random_sentence)

    return random_articles

def predict_news(random_articles):
    # Preprocess the articles
    cleaned_articles = [preprocess(article) for article in random_articles]
    cleaned_joined = [" ".join(article) for article in cleaned_articles]
    sequences = tokenizer.texts_to_sequences(cleaned_joined)
    padded_sequences = pad_sequences(sequences, maxlen=40, padding='post', truncating='post')
    predictions = model.predict(padded_sequences)
    predicted_labels = [1 if pred > 0.5 else 0 for pred in predictions]
    return predicted_labels
random_news = generate_random_news(num_articles=1500)
predictions = predict_news(random_news)
for article, prediction in zip(random_news, predictions):
    print(f"Article: {article}\nPrediction: {'Real News' if prediction == 1 else 'Fake News'}\n")

Code for XAI

In [ ]:
import shap
import pandas as pd
import numpy as np

explainer = shap.Explainer(model, x_train)  
subset_x_test = x_test[:10]
shap_values = explainer(subset_x_test)
shap.summary_plot(shap_values, subset_x_test)

import seaborn as sns

feature_names = tokenizer.index_word
num_features = min(len(feature_names), padded_sequences.shape[1])
feature_names_list = ['Padding'] + [feature_names[i] for i in range(1, num_features)]
shap_df = pd.DataFrame(shap_values.values, columns=feature_names_list)
mean_shap_values = np.abs(shap_df).mean(axis=0)
sorted_feature_indices = mean_shap_values.sort_values(ascending=False).index[:20]  # Get top 20 important features
top_feature_names = []
top_importances = []
print("Top Features shown in the SHAP summary plot with their importance:")
for feature in sorted_feature_indices:
    if feature == 'Padding':
        index = 0
        importance_value = mean_shap_values[index]
        top_feature_names.append('Padding')
        top_importances.append(importance_value)
        print(f"Feature Index: {index}, Feature Name: Padding, Importance: {importance_value}")
    else:
        feature_index = [k for k, v in feature_names.items() if v == feature]
        if feature_index:
            index = feature_index[0]
            importance_value = mean_shap_values[index]
            top_feature_names.append(feature)
            top_importances.append(importance_value)
            print(f"Feature Index: {index}, Feature Name: {feature}, Importance: {importance_value}")
        else:
            print(f"Feature '{feature}' not found in feature names.")

def plot_shap_values(feature_names, importances):
    shap_df = pd.DataFrame({
        'Feature Name': feature_names,
        'Importance': importances
    })

    shap_df = shap_df.sort_values(by='Importance', ascending=False)
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Importance', y='Feature Name', data=shap_df)

    plt.title('Features Shown in Summary Plot by SHAP')
    plt.xlabel('Feature Impact on Prediction')
    plt.ylabel('Feature Names')

    plt.show()

plot_shap_values(top_feature_names, top_importances)

Contract Deployment

In [ ]:
const FakeNewsDetection = artifacts.require("FakeNewsDetection");

module.exports = function(deployer){
    deployer.deploy(FakeNewsDetection);
}

Smart Contract

In [ ]:
// SPDX-License-Identifier: MIT
pragma solidity ^0.5.16;

contract FakeNewsDetection {
    struct News {
        uint id;
        string content;
        address submitter;
        bool isVerified;
        bool isFake;
        uint credibilityPoints;
    }

    mapping(uint => News) public newsRecords;
    mapping(address => uint) public userCredibility;
    uint public newsCount;

    event NewsSubmitted(uint id, string content, address indexed submitter);
    event NewsVerified(uint id, bool isFake, address indexed verifier, uint credibilityPoints);

    constructor() public{
        newsCount = 0;
    }

    function submitNews(string memory _content) public {
        newsCount++;
        newsRecords[newsCount] = News(newsCount, _content, msg.sender, false, false, 0);
        emit NewsSubmitted(newsCount, _content, msg.sender);
    }

    function verifyNews(uint _newsId, bool _isFake) public {
        require(_newsId != 0 && _newsId <= newsCount, "Invalid news ID");
        require(!newsRecords[_newsId].isVerified, "News already verified");
        newsRecords[_newsId].isFake = _isFake;
        newsRecords[_newsId].isVerified = true;
        uint credibilityPoints = _isFake ? 10 : 5;
        newsRecords[_newsId].credibilityPoints = credibilityPoints;
        userCredibility[msg.sender] = userCredibility[msg.sender] + credibilityPoints;

        emit NewsVerified(_newsId, _isFake, msg.sender, credibilityPoints);
    }

    function getNews(uint _newsId) public view returns (string memory, bool, bool, uint) {
        require(_newsId != 0 && _newsId <= newsCount, "Invalid news ID");
        News memory newsItem = newsRecords[_newsId];
        return (newsItem.content, newsItem.isVerified, newsItem.isFake, newsItem.credibilityPoints);
    }

    function getUserCredibility(address _user) public view returns (uint) {
        return userCredibility[_user];
    }
}

Contract Testing

In [ ]:
const FakeNewsDetection = artifacts.require('FakeNewsDetection');

contract('FakeNewsDetection', (accounts) => {
  let contractInstance;

  before(async () => {
    contractInstance = await FakeNewsDetection.deployed();
  });

  describe('Deployment', () => {
    it('should deploy successfully', async () => {
      assert.notEqual(contractInstance.address, undefined, 'Contract address is undefined');
    });
  });

  describe('News Submission', () => {
    it('should allow submitting news', async () => {
      await contractInstance.submitNews('Breaking News!', { from: accounts[0] });
      const news = await contractInstance.newsRecords(1);
      assert.equal(news.content, 'Breaking News!', 'News content mismatch');
    });
  });
  describe('News Verification', () => {
    it('should allow a verifier to verify news as fake and award credibility points', async () => {
      const tx = await contractInstance.verifyNews(1, true, { from: accounts[1] });

      const news = await contractInstance.newsRecords(1);
      expect(news.isVerified).to.be.true;
      expect(news.isFake).to.be.true;
      expect(news.credibilityPoints.toNumber()).to.equal(10);

      const verifierCredibility = await contractInstance.getUserCredibility(accounts[1]);
      expect(verifierCredibility.toNumber()).to.equal(10);

      expect(tx.logs[0].event).to.equal('NewsVerified');
      expect(tx.logs[0].args.isFake).to.be.true;
    });

    it('should allow a verifier to verify news as not fake and award credibility points', async () => {
      await contractInstance.submitNews('Another news content', { from: accounts[0] });
      const newsCount = await contractInstance.newsCount();
      expect(newsCount.toNumber()).to.equal(2);

      const tx = await contractInstance.verifyNews(2, false, { from: accounts[1] });

      const news = await contractInstance.newsRecords(2);
      expect(news.isVerified).to.be.true;
      expect(news.isFake).to.be.false;
      expect(news.credibilityPoints.toNumber()).to.equal(5);

      const verifierCredibility = await contractInstance.getUserCredibility(accounts[1]);
      expect(verifierCredibility.toNumber()).to.equal(15);

      expect(tx.logs[0].event).to.equal('NewsVerified');
      expect(tx.logs[0].args.isFake).to.be.false;
    });
  });
});

Web3.js code for interaction with the model

In [ ]:
const Web3 = require("web3");
const axios = require("axios"); 
const contractABI = require("D:\Studies\Blockchain\Truffle\FakeNews\FakeNews.json").abi; 
const contractAddress = "0xb06111b8AdDC6d1bdC7bF771F036472AbbfF67b1"; 
const web3 = new Web3("http://127.0.0.1:7545"); 
const contract = new web3.eth.Contract(contractABI, contractAddress);
const account = "0xb06111b8AdDC6d1bdC7bF771F036472AbbfF67b1"; 
const mlApiUrl = "http://127.0.0.1:5000/predict";
contract.events
  .NewsSubmitted()
  .on("data", async (event) => {
    console.log("News Submitted Event Detected:", event);
    const { newsId, content } = event.returnValues;
    console.log(`Processing news with ID: ${newsId}, Content: ${content}`);
    try {
      const response = await axios.post(mlApiUrl, { text: content });
      if (response.data && response.data.prediction !== undefined) {
        const prediction = response.data.prediction; 
        console.log(`Prediction received: ${prediction ? "Real" : "Fake"}`);
        await contract.methods
          .verifyNews(newsId, prediction === 1) 
          .send({ from: account, gas: 300000 });
        console.log(`News ID ${newsId} verification updated on the blockchain.`);
      } else {
        console.error("Invalid response from ML model:", response.data);
      }
    } catch (error) {
      console.error("Error processing news with ML model:", error.message);
    }
  })
  .on("error", (error) => {
    console.error("Error listening to NewsSubmitted events:", error.message);
  });
console.log("Listening for NewsSubmitted events...");
